# Validations

## Bmh knockout

Source: https://github.com/ntumitolab/MitoRetroDynamics/blob/main/validation.ipynb

Initial protein concentrations: Get expression levels: uses data from [GSE102475](https://github.com/NTUMitoLab/RetroSignalModel.jl?organization=NTUMitoLab&organization=NTUMitoLab#differential-gene-expression).

In [ ]:
using Plots
using StatsPlots
using ModelingToolkit
using DifferentialEquations
using RetroSignalModel
import RetroSignalModel as rs

Let's try one set of params and solve the steady-state solution:

In [ ]:
proteinwt = rs.STRESSED
proteinΔbmh = Dict(proteinwt..., rs.ΣBmh=>1e-4)

@named sys = RtgMTK(rs.ONE_SIGNAL)
u0 = rs.resting_u0(sys)

ks = load_parameters("solution_rtgM4.csv")[1]
params = merge(proteinwt, ks, Dict(rs.mul_S => 0.1))
prob = SteadyStateProblem(sys, u0, params)
sol = solve(prob, DynamicSS(Rodas5()))

Considering a combination of Wild-type vs. Bmh knockout, and Low damage signal = `0.1` vs. High = `0.9`.

In [ ]:
idxmulS = findfirst(isequal(rs.mul_S), parameters(sys))
idxBmh = findfirst(isequal(rs.ΣBmh), parameters(sys))

In [ ]:
r3nc_wt = map(0.0:0.02:1.0) do s
    p = copy(prob.p)
    p[idxmulS] = s
    sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
    rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
end

In [ ]:
r3nc_Δb = map(0.0:0.02:1.0) do s
    p = copy(prob.p)
    p[idxmulS] = s
    p[idxBmh] = 1e-4
    sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
    rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
end

In [ ]:
plot(0.0:0.02:1.0, [r3nc_wt r3nc_Δb], label=["Wild Type" "ΔBmh"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 5.0),  xlabel="Damage Signal", ylabel="Ratio of nucleus to cytosol RTG3")

## Rapamycin and glutamate

`kn2M` was decreased ten times to simulated enhanced binding of Mks and Rtg2 upon Rapamycin / glutamate addition.

In [ ]:
idxkn2M = findfirst(isequal(rs.kn2M), parameters(sys))
r3nc_ra = map(0.0:0.02:1.0) do s
    p = copy(prob.p)
    p[idxmulS] = s
    p[idxkn2M] /= 10
    sol = solve(remake(prob, p=p), DynamicSS(Rodas5()))
    rs.rtg3_nucleus(sol) / rs.rtg3_cytosol(sol)
end

plot(0.0:0.02:1.0, [r3nc_wt r3nc_ra], label=["Wild Type" "Rapamycin / Glutamate"], lw=2, legend=:right, 
    xlims=(0.0, 1.0), ylims=(0.0, 5.0),  xlabel="Damage Signal", ylabel="Ratio of nucleus to cytosol RTG3")